In [1]:
#sys.path.append('/Users/gabriellachronis/Box Sync/src/features_in_context')
import sys

sys.path.append('..')
sys.path.append("../src/")
import torch
from bert import *
from feature_data import *
from multiprototype import *
from scipy.stats import spearmanr

from classifier_main import *
import csv

bert = BERTBase()

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file /Users/gabriellachronis/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/9m/vzvx58rs51v_x5nm620fz4xr0000gn/T/tmpieymq7bg
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 3

Okay so you want to take each of these trained models and get the same test set that was used to train them. that is the first task. get those words.
and then get the features for those words

In [2]:
feature_norms = McRaeFeatureNorms('../data/external/mcrae/CONCS_FEATS_concstats_brm/concepts_features-Table1.csv')
layer = 8
clusters = 5
embedding_file = '../data/processed/multipro_embeddings/layer'+ str(layer) + 'clusters' + str(clusters) + '.txt'
embs = read_multiprototype_embeddings(embedding_file, layer=layer, num_clusters=clusters)

train, dev, test = prepare_data(feature_norms, embs)

Read in 12348 vectors of size 5 X 768
Starting with 433 training words


In [3]:
test

In [4]:
test_words = []
for word in test:
    test_words.append(word)

test_words

['coconut',
 'crayon',
 'calf',
 'yacht',
 'potato',
 'pier',
 'blackbird',
 'spear',
 'microscope',
 'cow',
 'projector',
 'drill',
 'nylons',
 'bus',
 'shelves',
 'cake',
 'crowbar',
 'bullet',
 'lamb',
 'stereo',
 'stool_(furniture)',
 'rocket',
 'belt',
 'violin',
 'walnut',
 'clock',
 'alligator',
 'tongs',
 'taxi',
 'bucket',
 'ambulance',
 'swimsuit',
 'pants',
 'camisole',
 'mat',
 'bread',
 'door',
 'bull',
 'knife',
 'coyote',
 'cod',
 'grapefruit',
 'lobster',
 'strainer',
 'helicopter',
 'cauliflower',
 'tank_(army)',
 'house',
 'chisel',
 'blueberry',
 'bison',
 'robe',
 'cupboard',
 'card_(greeting)']

In [26]:
len(test_words)

54

okay now we need to get the most frequent features for all words in features

In [5]:
num_features = len(feature_norms.feature_map)
feature_counts = Counter()


for word in feature_norms.feature_norms:
    features = feature_norms.get_features(word)
    for feature in features:
        feature_counts[feature] += 1
        #add feature to counter

feature_counts.most_common(10)

[('made_of_metal', 133),
 ('is_small', 121),
 ('is_large', 106),
 ('an_animal', 99),
 ('is_long', 81),
 ('made_of_wood', 79),
 ('is_edible', 78),
 ('is_round', 76),
 ('different_colours', 58),
 ('is_brown', 54)]

In [6]:
# order all mcrae features by frequency
most_frequent_features_in_order = feature_counts.most_common(num_features)
most_frequent_features_in_order = [x for (x,y) in most_frequent_features_in_order]

In [7]:
most_frequent_features_in_order[:50]

['made_of_metal',
 'is_small',
 'is_large',
 'an_animal',
 'is_long',
 'made_of_wood',
 'is_edible',
 'is_round',
 'different_colours',
 'is_brown',
 'is_white',
 'made_of_plastic',
 'is_black',
 'has_4_legs',
 'is_green',
 'beh_-_flies',
 'has_wings',
 'has_legs',
 'has_a_handle',
 'a_mammal',
 'has_a_tail',
 'a_bird',
 'beh_-_eats',
 'has_feathers',
 'has_a_beak',
 'is_dangerous',
 'is_loud',
 'clothing',
 'is_fast',
 'used_for_transportation',
 'beh_-_swims',
 'tastes_good',
 'lives_in_water',
 'beh_-_lays_eggs',
 'found_in_kitchens',
 'a_fruit',
 'a_weapon',
 'a_vegetable',
 'is_heavy',
 'is_yellow',
 'is_soft',
 'hunted_by_people',
 'is_red',
 'tastes_sweet',
 'has_wheels',
 'is_hard',
 'has_fur',
 'is_electrical',
 'is_expensive',
 'has_seeds']

In [8]:
most_frequent_features_in_order_by_index = [feature_norms.feature_map.index_of(x) for x in most_frequent_features_in_order]
most_frequent_features_in_order_by_index

[17,
 66,
 16,
 24,
 25,
 207,
 140,
 100,
 170,
 235,
 40,
 137,
 67,
 313,
 23,
 9,
 10,
 355,
 153,
 267,
 30,
 417,
 618,
 418,
 419,
 37,
 7,
 372,
 12,
 14,
 32,
 149,
 29,
 420,
 425,
 93,
 163,
 139,
 53,
 103,
 150,
 309,
 46,
 99,
 380,
 187,
 262,
 426,
 571,
 97,
 98,
 317,
 94,
 161,
 168,
 199,
 0,
 121,
 141,
 154,
 155,
 174,
 222,
 105,
 134,
 354,
 371,
 847,
 50,
 102,
 124,
 143,
 230,
 328,
 362,
 511,
 22,
 130,
 171,
 336,
 4,
 6,
 110,
 200,
 363,
 365,
 1194,
 65,
 117,
 296,
 401,
 716,
 1029,
 45,
 96,
 136,
 332,
 422,
 621,
 647,
 680,
 848,
 1014,
 28,
 43,
 133,
 165,
 169,
 220,
 273,
 284,
 305,
 308,
 351,
 353,
 421,
 427,
 462,
 465,
 667,
 971,
 1268,
 11,
 92,
 113,
 160,
 208,
 310,
 312,
 409,
 623,
 720,
 818,
 849,
 941,
 945,
 1063,
 1182,
 26,
 69,
 106,
 111,
 125,
 166,
 357,
 373,
 375,
 539,
 829,
 977,
 1198,
 1,
 156,
 159,
 167,
 183,
 184,
 186,
 190,
 198,
 283,
 306,
 383,
 385,
 442,
 474,
 505,
 509,
 544,
 569,
 611,
 637,
 649,


Now you want to take a model, make predictions on the 50 test words and have

In [9]:
model = torch.load('../trained_models/model.ffnn.mc_rae_real.5k.50epochs.0.5dropout.lr1e-4.hsize300')

In [10]:
#predictions = [model.predict_top_n_features(word, num_features) for word in test]
predictions = [model.predict(word) for word in test]

In [11]:
print(test[0])
predictions[0][:10]

coconut


array([ 0.43001604, -0.21088235,  0.39697215,  0.09591179, -0.9964212 ,
        0.08740016,  0.22333442, -0.02269508,  0.07093224, -0.88892674],
      dtype=float32)

In [12]:
# #okay now what do you do with the prediction.... run the correlations with each predition and the frequent features
# xs = []
# ys = []

# i=0
# prediction = predictions[i]
# counter = Counter()
# for i  in range(0, len(prediction)):
#     feature_prediction = prediction[i]
#     feature_name = feature_norms.feature_map.get_object(i)
#     counter[feature_name] = feature_prediction

# #for prediction in predictions:
# in_order =  counter.most_common(num_features)
# features_in_order = [x for (x,y) in in_order]
# print(f


    
# correlation_analysis(test_words, feature_norms, predictions, most_frequent_features_in_order)

In [13]:
feature_norms.get_features('mat')

['found_by_doors',
 'used_for_wiping_feet',
 'used_for_welcoming',
 'found_in_bathrooms',
 'a_rug',
 'found_in_cars',
 'found_on_floors',
 'used_for_covering_floors',
 'is_small',
 'made_of_material',
 'made_of_rubber',
 'used_for_protecting_things',
 'used_for_wiping_shoes']

In [24]:
# def correlation_analysis(test_words, feature_norms, predictions, most_frequent_features_in_order):
#     xs = []
#     ys = []
#     for i in range(0,len(test_words)):
#         #print(test_words[i])
#         prediction = predictions[i]
#         counter = Counter()
#         for i  in range(0, len(prediction)):
#             feature_prediction = prediction[i]
#             feature_name = feature_norms.feature_map.get_object(i)
#             counter[feature_name] = feature_prediction

#         #for prediction in predictions:
#         in_order =  counter.most_common(num_features)
#         features_in_order = [x for (x,y) in in_order]

#         # run correlation
#         xs = xs + most_frequent_features_in_order
#         ys = ys + features_in_order
#         #coef, p = spearmanr(most_frequent_features_in_order, features_in_order)
#         #print(coef, p)
#         #corrs.append((coef,p))
        

#     coef, p = spearmanr(xs, ys)
#     return coef, p

# this is correlation of each result, averaged
def correlation_analysis(test_words, feature_norms, predictions, most_frequent_features_in_order, top_n=-1):
    """
    this function measures the correlation between the most frequent features in the dataset
    and the features predicted by the model
    
    :test_words:
    :feature_norms:
    :predictions:
    :most_frequent_features_in_order:
    :top_n: int  optional argument what cuts off the features to look at rank correlation for the top n
    """
    
    # you need to convert frequent features to a ranked list
    most_frequent_features_in_order_by_index = [feature_norms.feature_map.index_of(x) for x in most_frequent_features_in_order]
    
    corrs = []
    #for prediction in predictions:
    for i in range(0,len(test_words)):
        
        #print(test_words[i])
        prediction = predictions[i]
        #print(prediction)
        counter = Counter()
        for j  in range(0, len(prediction)):
            feature_prediction = prediction[j]
            feature_name = feature_norms.feature_map.get_object(j)
            counter[feature_name] = feature_prediction
        
        #print(counter)
        #raise Exception("fejiowfe")

        num_features = len(feature_norms.feature_map)
        in_order =  counter.most_common(num_features)
        features_in_order = [x for (x,y) in in_order]
        features_in_order_by_index = [feature_norms.feature_map.index_of(x) for x in features_in_order]

        # run correlation
        #print(test_words[i])
        print(most_frequent_features_in_order_by_index[:top_n])
        print(features_in_order_by_index[:top_n])
        coef, p = spearmanr(most_frequent_features_in_order_by_index[:top_n], features_in_order_by_index[:top_n])
        #print(coef, p)
        corrs.append((coef,p))
        
        average_correlation = np.average([coef for (coef,p) in corrs])
        #print(corrs)
        ps = [p for (corr,p) in corrs]
    return average_correlation, ps



for top_n in [10, 20, -1]:
    print("correlation between top %s gold features and predicted features for test set:" % top_n)
    avg_cor, ps = correlation_analysis(test_words, feature_norms, predictions, most_frequent_features_in_order, top_n)
    print(avg_cor)
    print(np.average(ps))


correlation between top 10 gold features and predicted features for test set:
[17, 66, 16, 24, 25, 207, 140, 100, 170, 235]
[93, 103, 140, 94, 23, 149, 150, 202, 99, 25]
[17, 66, 16, 24, 25, 207, 140, 100, 170, 235]
[613, 153, 601, 17, 207, 2114, 25, 168, 155, 588]
[17, 66, 16, 24, 25, 207, 140, 100, 170, 235]
[24, 618, 313, 626, 716, 355, 30, 1069, 16, 267]
[17, 66, 16, 24, 25, 207, 140, 100, 170, 235]
[467, 14, 190, 470, 16, 207, 468, 1605, 757, 66]
[17, 66, 16, 24, 25, 207, 140, 100, 170, 235]
[139, 23, 103, 140, 93, 100, 50, 362, 102, 141]
[17, 66, 16, 24, 25, 207, 140, 100, 170, 235]
[17, 589, 207, 590, 16, 591, 284, 592, 66, 25]
[17, 66, 16, 24, 25, 207, 140, 100, 170, 235]
[417, 9, 418, 10, 419, 66, 618, 67, 24, 420]
[17, 66, 16, 24, 25, 207, 140, 100, 170, 235]
[17, 154, 163, 207, 153, 25, 151, 37, 161, 160]
[17, 66, 16, 24, 25, 207, 140, 100, 170, 235]
[2359, 17, 130, 2360, 25, 153, 154, 100, 2361, 160]
[17, 66, 16, 24, 25, 207, 140, 100, 170, 235]
[24, 618, 313, 716, 626, 355

[17, 66, 16, 24, 25, 207, 140, 100, 170, 235, 40, 137, 67, 313, 23, 9, 10, 355, 153, 267]
[1792, 121, 25, 372, 328, 511, 336, 170, 93, 518, 150, 117, 99, 555, 100, 50, 98, 818, 439, 97]
[17, 66, 16, 24, 25, 207, 140, 100, 170, 235, 40, 137, 67, 313, 23, 9, 10, 355, 153, 267]
[150, 207, 375, 170, 17, 323, 66, 376, 25, 328, 810, 811, 812, 813, 814, 65, 324, 67, 100, 325]
[17, 66, 16, 24, 25, 207, 140, 100, 170, 235, 40, 137, 67, 313, 23, 9, 10, 355, 153, 267]
[140, 910, 103, 50, 149, 40, 150, 911, 409, 714, 912, 913, 914, 139, 915, 100, 23, 187, 917, 916]
[17, 66, 16, 24, 25, 207, 140, 100, 170, 235, 40, 137, 67, 313, 23, 9, 10, 355, 153, 267]
[17, 207, 696, 1212, 1139, 1213, 1207, 1138, 1214, 1208, 1210, 1406, 100, 1407, 1408, 66, 401, 2289, 1215, 1409]
[17, 66, 16, 24, 25, 207, 140, 100, 170, 235, 40, 137, 67, 313, 23, 9, 10, 355, 153, 267]
[24, 16, 313, 30, 267, 355, 417, 618, 262, 235, 626, 1525, 418, 923, 716, 317, 9, 680, 309, 419]
[17, 66, 16, 24, 25, 207, 140, 100, 170, 235, 40, 

[17, 66, 16, 24, 25, 207, 140, 100, 170, 235, 40, 137, 67, 313, 23, 9, 10, 355, 153, 267, 30, 417, 618, 418, 419, 37, 7, 372, 12, 14, 32, 149, 29, 420, 425, 93, 163, 139, 53, 103, 150, 309, 46, 99, 380, 187, 262, 426, 571, 97, 98, 317, 94, 161, 168, 199, 0, 121, 141, 154, 155, 174, 222, 105, 134, 354, 371, 847, 50, 102, 124, 143, 230, 328, 362, 511, 22, 130, 171, 336, 4, 6, 110, 200, 363, 365, 1194, 65, 117, 296, 401, 716, 1029, 45, 96, 136, 332, 422, 621, 647, 680, 848, 1014, 28, 43, 133, 165, 169, 220, 273, 284, 305, 308, 351, 353, 421, 427, 462, 465, 667, 971, 1268, 11, 92, 113, 160, 208, 310, 312, 409, 623, 720, 818, 849, 941, 945, 1063, 1182, 26, 69, 106, 111, 125, 166, 357, 373, 375, 539, 829, 977, 1198, 1, 156, 159, 167, 183, 184, 186, 190, 198, 283, 306, 383, 385, 442, 474, 505, 509, 544, 569, 611, 637, 649, 722, 757, 877, 889, 923, 929, 939, 978, 1082, 1956, 87, 116, 158, 173, 177, 205, 226, 238, 242, 294, 302, 368, 423, 437, 439, 445, 478, 504, 508, 519, 522, 549, 568, 626, 6

[17, 66, 16, 24, 25, 207, 140, 100, 170, 235, 40, 137, 67, 313, 23, 9, 10, 355, 153, 267, 30, 417, 618, 418, 419, 37, 7, 372, 12, 14, 32, 149, 29, 420, 425, 93, 163, 139, 53, 103, 150, 309, 46, 99, 380, 187, 262, 426, 571, 97, 98, 317, 94, 161, 168, 199, 0, 121, 141, 154, 155, 174, 222, 105, 134, 354, 371, 847, 50, 102, 124, 143, 230, 328, 362, 511, 22, 130, 171, 336, 4, 6, 110, 200, 363, 365, 1194, 65, 117, 296, 401, 716, 1029, 45, 96, 136, 332, 422, 621, 647, 680, 848, 1014, 28, 43, 133, 165, 169, 220, 273, 284, 305, 308, 351, 353, 421, 427, 462, 465, 667, 971, 1268, 11, 92, 113, 160, 208, 310, 312, 409, 623, 720, 818, 849, 941, 945, 1063, 1182, 26, 69, 106, 111, 125, 166, 357, 373, 375, 539, 829, 977, 1198, 1, 156, 159, 167, 183, 184, 186, 190, 198, 283, 306, 383, 385, 442, 474, 505, 509, 544, 569, 611, 637, 649, 722, 757, 877, 889, 923, 929, 939, 978, 1082, 1956, 87, 116, 158, 173, 177, 205, 226, 238, 242, 294, 302, 368, 423, 437, 439, 445, 478, 504, 508, 519, 522, 549, 568, 626, 6

In [25]:
spearmanr([17, 66, 16, 24, 25, 207, 140, 100, 170, 235],
          [93, 103, 140, 94, 23, 149, 150, 202, 99, 25]

)

SpearmanrResult(correlation=0.13939393939393938, pvalue=0.7009318849100584)

In [15]:
models = [
    '../trained_models/model.ffnn.mc_rae_real.5k.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.ffnn.mc_rae_real.1k.50epochs.0.5dropout.lr1e-4.hsize300',
    #'../trained_models/model.ffnn.mc_rae_real.glove.50epochs.0.5dropout.lr1e-4.hsize300',
    '../trained_models/model.plsr.mc_rae_real.5k.100components.500max_iters',
    '../trained_models/model.plsr.mc_rae_real.1k.50components.500max_iters',
    #'../trained_models/model.plsr.mc_rae_real.glove.100components.500max_iters',
    '../trained_models/model.modabs.mc_rae_real.5k.mu1_1.mu2_0.1.mu3_0.001.mu4_5.nnk_4',
    '../trained_models/model.modabs.mc_rae_real.1k.mu1_1.mu2_0.1.mu3_1e-07.mu4_10.nnk_4',
    #'../trained_models/model.modabs.mc_rae_real.glove.mu1_1.mu2_0.1.mu3_1e-07.mu4_5.nnk_5',
]


In [16]:
for model in models:
    print(model)
    model = torch.load(model)
    predictions = predictions = [model.predict(word) for word in test]
    
    
    for top_n in [10, 20, -1]:
        print("correlation between top %s gold features and predicted features for test set:" % top_n)
        avg_corr, ps = correlation_analysis(test_words, feature_norms, predictions, most_frequent_features_in_order, top_n)
        print(avg_corr)
        print(np.average(ps))

    print("")

../trained_models/model.ffnn.mc_rae_real.5k.50epochs.0.5dropout.lr1e-4.hsize300
correlation between top 10 gold features and predicted features for test set:
0.15578002244668912
0.4927465213997824
correlation between top 20 gold features and predicted features for test set:
0.04291283764967976
0.5366053945539291
correlation between top -1 gold features and predicted features for test set:
0.004247399546499077
0.29423790737980604

../trained_models/model.ffnn.mc_rae_real.1k.50epochs.0.5dropout.lr1e-4.hsize300
correlation between top 10 gold features and predicted features for test set:
0.2101010101010101
0.4939868571944259
correlation between top 20 gold features and predicted features for test set:
0.11720969089390142
0.5065730874179164
correlation between top -1 gold features and predicted features for test set:
0.0032101338304500507
0.24529945513443605

../trained_models/model.plsr.mc_rae_real.5k.100components.500max_iters
correlation between top 10 gold features and predicted featur

In [17]:
test_words
len(test_words)

54

In [18]:
for i in range(0,len(test_words)):
    print(test_words[i])

coconut
crayon
calf
yacht
potato
pier
blackbird
spear
microscope
cow
projector
drill
nylons
bus
shelves
cake
crowbar
bullet
lamb
stereo
stool_(furniture)
rocket
belt
violin
walnut
clock
alligator
tongs
taxi
bucket
ambulance
swimsuit
pants
camisole
mat
bread
door
bull
knife
coyote
cod
grapefruit
lobster
strainer
helicopter
cauliflower
tank_(army)
house
chisel
blueberry
bison
robe
cupboard
card_(greeting)


In [19]:
town1 = ['dog', 'cat', 'possum', 'fish', 'pig', 'degu']
town11 = [1, 2, 3, 4, 5, 6]

town2 = ['dog', 'cat', 'degu', 'fish', 'cow', 'goat']
town22 = [1,2,6,4,7,8]

spearmanr(town1, town2)

SpearmanrResult(correlation=-0.028571428571428574, pvalue=0.9571545189504373)

In [20]:
spearmanr(town11, town22)

SpearmanrResult(correlation=0.942857142857143, pvalue=0.004804664723032055)

In [21]:
town22 = [9,10,6,4,7,8]
spearmanr(town11, town22)

SpearmanrResult(correlation=-0.4285714285714286, pvalue=0.3965014577259474)

In [22]:
class FrequencyClassifier(object):
    """
    Person classifier that takes counts of how often a word was observed to be the positive and negative class
    in training, and classifies as positive any tokens which are observed to be positive more than negative.
    Unknown tokens or ties default to negative.
    Attributes:
        pos_counts: how often each token occurred with the label 1 in training
        neg_counts: how often each token occurred with the label 0 in training
    """
    def __init__(self, feature_norms):
        vecs = []
        self.feature_norms = feature_norms
        words = self.feature_norms.feature_norms.keys()
        for word in words:
            vec = feature_norms.get_feature_vector(word)
            vecs.append(vec)
            
        avg_features = np.average(vecs, axis=0)
        print(avg_features)

        ind = np.argpartition(avg_features, -10)[-10:]
        feats = []
        for i in ind:
            feat = feature_norms.feature_map.get_object(i)
            feats.append(feat)
        print(feats)
        
        self.prediction = avg_features
        

    def predict(self, word):
        return self.prediction

In [23]:
cs = FrequencyClassifier(feature_norms)

[0.74306839 0.19963031 0.09057301 ... 0.012939   0.012939   0.00924214]
['is_round', 'is_green', 'is_long', 'is_edible', 'a_bird', 'is_large', 'an_animal', 'is_small', 'made_of_wood', 'made_of_metal']
